### **Importaciones**

In [ ]:
import pyodbc
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

### **Conectar los datos**

In [1]:
import pandas as pd

# Ruta al archivo CSV (ajusta esta ruta si es diferente)
csv_file = './Data/Caso_02_GD.csv'  # Ruta relativa desde donde estás ejecutando el script

# Cargar el archivo CSV en un DataFrame de pandas
df = pd.read_csv(csv_file)

# Mostrar las primeras filas del DataFrame para verificar que se cargó correctamente
df.head()


,ID,Genero,Casado,Edad,Graduado,Profesion,Experiencia_laboral,Gastoscore,Family_Size,anno_nacimiento,Generacion,Ingresos_anuales_brutos,Provincia,Digital_encuesta,Gasto_medio_mensual_cuota,Abandono,Dias_cliente,Campanna_anno
0,CLI10,F,Si,67,Si,Ingeniero,+20annos,Bajo,3,1957,Generacion X,93442,Segovia,1385,80,1,195,NaN
1,CLI100,M,Si,64,Si,Medico,+20annos,Medio,2,1960,Generacion X,99774,Alicante,1052,50,1,360,5.0
2,CLI1000,M,Si,65,Si,Artista,+20annos,Medio,3,1959,Generacion X,36756,Toledo,1717,30,0,407,3.0
3,CLI10000,M,No,33,No,Funcionario,10-20annos,Bajo,4,1991,Millennials,24616,Leon,1606,30,0,544,4.0
4,CLI10001,M,Si,52,No,Otros,+20annos,Medio,2,1972,Generacion X,29834,LLeida,1346,30,1,170,NaN


In [2]:
# Cuales son las vraiables numericas 
numericas = df.select_dtypes(include=[np.number]).columns.tolist()
# pero que se printee cuales son 
print("Las variables numericas son: ", numericas)

NameError: name 'np' is not defined

In [ ]:
# Quiero ver cual es el valor mas alto y cual es el mas bajo de las variables numericas, de TODAS que no se corte 
for col in numericas:
    print(f"Variable: {col}")
    print(f"Valor más bajo: {df[col].min()}")
    print(f"Valor más alto: {df[col].max()}")
    print()

Variable: Edad
Valor más bajo: 20
Valor más alto: 98

Variable: Family_Size
Valor más bajo: 1
Valor más alto: 9

Variable: anno_nacimiento
Valor más bajo: 1926
Valor más alto: 2004

Variable: Ingresos_anuales_brutos
Valor más bajo: 12003
Valor más alto: 129963

Variable: Digital_encuesta
Valor más bajo: 7
Valor más alto: 15215

Variable: Gasto_medio_mensual_cuota
Valor más bajo: 30
Valor más alto: 200

Variable: Abandono
Valor más bajo: 0
Valor más alto: 1

Variable: Dias_cliente
Valor más bajo: 30
Valor más alto: 600

Variable: Campanna_anno
Valor más bajo: 1.0
Valor más alto: 5.0



In [ ]:
print(df.dtypes)

ID                            object
Genero                        object
Casado                        object
Edad                           int64
Graduado                      object
Profesion                     object
Experiencia_laboral           object
Gastoscore                    object
Family_Size                    int64
anno_nacimiento                int64
Generacion                    object
Ingresos_anuales_brutos        int64
Provincia                     object
Digital_encuesta               int64
Gasto_medio_mensual_cuota      int64
Abandono                       int64
Dias_cliente                   int64
Campanna_anno                float64
dtype: object


In [ ]:
numerical_columns = df.select_dtypes(include=['float64', 'int64']).columns

# Mostrar estadísticas descriptivas para ver los rangos y las diferencias
print(df[numerical_columns].describe())

# Definir un umbral de diferencia para identificar las columnas que deben normalizarse
threshold = 1000  # Por ejemplo, si la diferencia entre el valor máximo y mínimo es mayor que 1000, normalizamos

# Identificar columnas que tienen una gran diferencia de escala
columns_to_normalize = []

for column in numerical_columns:
    min_value = df[column].min()
    max_value = df[column].max()
    
    if max_value - min_value > threshold:  # Si la diferencia es mayor al umbral
        columns_to_normalize.append(column)

print("Columnas que podrían necesitar normalización:", columns_to_normalize)

               Edad   Family_Size  anno_nacimiento  Ingresos_anuales_brutos  \
count  10806.000000  10806.000000     10806.000000             10806.000000   
mean      47.920415      2.764390      1976.075236             56113.470387   
std       18.432100      1.548241        18.431804             35458.081118   
min       20.000000      1.000000      1926.000000             12003.000000   
25%       33.000000      2.000000      1966.000000             24986.250000   
50%       45.000000      2.000000      1979.000000             43628.000000   
75%       58.000000      4.000000      1991.000000             86678.500000   
max       98.000000      9.000000      2004.000000            129963.000000   

       Digital_encuesta  Gasto_medio_mensual_cuota      Abandono  \
count      10806.000000               10806.000000  10806.000000   
mean        1597.302610                  45.285027      0.297983   
std         2135.573309                  28.674032      0.457393   
min            7

#### Normalizar las columnas numericas

#### Mapear 


In [ ]:
df

,ID,Genero,Casado,Edad,Graduado,Profesion,Experiencia_laboral,Gastoscore,Family_Size,anno_nacimiento,Generacion,Ingresos_anuales_brutos,Provincia,Digital_encuesta,Gasto_medio_mensual_cuota,Abandono,Dias_cliente,Campanna_anno
0,CLI10,F,Si,67,Si,Ingeniero,+20annos,Bajo,3,1957,Generacion X,93442,Segovia,1385,80,1,195,NaN
1,CLI100,M,Si,64,Si,Medico,+20annos,Medio,2,1960,Generacion X,99774,Alicante,1052,50,1,360,5.0
2,CLI1000,M,Si,65,Si,Artista,+20annos,Medio,3,1959,Generacion X,36756,Toledo,1717,30,0,407,3.0
3,CLI10000,M,No,33,No,Funcionario,10-20annos,Bajo,4,1991,Millennials,24616,Leon,1606,30,0,544,4.0
4,CLI10001,M,Si,52,No,Otros,+20annos,Medio,2,1972,Generacion X,29834,LLeida,1346,30,1,170,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10801,CLI9995,F,Si,76,No,Servicios,+20annos,Alto,2,1948,Generacion X,33389,Toledo,1424,30,1,106,NaN
10802,CLI9996,F,No,44,Si,Funcionario,+20annos,Bajo,1,1980,Generacion X,76067,Cordoba,1514,50,0,257,NaN
10803,CLI9997,M,Si,47,Si,Ejecutivo,+20annos,Alto,3,1977,Generacion X,61300,Lugo,1385,80,1,347,5.0
10804,CLI9998,M,Si,40,Si,Artista,10-20annos,Alto,3,1984,Millennials,106015,Cuenca,1974,30,0,582,1.0


In [ ]:
# Ver valores únicos en la columna 'campanna_anno'
provincias_unicas = df['Provincia'].unique()
provincias_unicas

array(['Segovia', 'Alicante', 'Toledo', 'Leon', 'LLeida', 'Ciudad Real',
       'i\x81vila', 'Guipiºzcoa', 'Vizcaya', 'Orense', 'Castellon',
       'Pontevedra', 'Almeri\xada', 'Valladolid', 'Jaen', 'Lugo',
       'Santa Cruz de Tenerife', 'Albacete', 'Huelva', 'Salamanca',
       'Ceuta', 'Baleares', 'Gerona', 'Barcelona', 'Sevilla',
       'Guadalajara', 'Soria', 'Caceres', 'Madrid', 'Cordoba', 'Burgos',
       'Cadiz', 'Navarra', 'Melilla', 'Granada', 'Badajoz', 'Valencia',
       'Tarragona', 'La Corui±a', 'Las Palmas', 'Murcia', 'Palencia',
       'La Rioja', 'Teruel', 'Zamora', 'Zaragoza', 'i\x81lava', 'Cuenca',
       'Huesca', 'Malaga'], dtype=object)

In [ ]:
# Diccionario de mapeo de provincias a comunidades autónomas
mapa_provincias = {
    'i\x81vila': 'Castilla y León',
    'Álava': 'País Vasco',
    'i\x81lava': 'País Vasco',
    'Albacete': 'Castilla-La Mancha',
    'Alicante': 'Comunidad Valenciana',
    'Almería': 'Andalucía',
    'Ávila': 'Castilla y León',
    'Badajoz': 'Extremadura',
    'Guipiºzcoa': 'País Vasco',
    'Baleares': 'Islas Baleares',
    'Barcelona': 'Cataluña',
    'Burgos': 'Castilla y León',
    'Caceres': 'Extremadura',
    'Cadiz': 'Andalucía',
    'Castellon': 'Comunidad Valenciana',
    'Ciudad Real': 'Castilla-La Mancha',
    'Cordoba': 'Andalucía',
    'La Coruña': 'Galicia',
    'Cuenca': 'Castilla-La Mancha',
    'Gerona': 'Cataluña',
    'Granada': 'Andalucía',
    'Guadalajara': 'Castilla-La Mancha',
    'Guipúzcoa': 'País Vasco',
    'Huelva': 'Andalucía',
    'Huesca': 'Aragón',
    'Jaen': 'Andalucía',
    'Leon': 'Castilla y León',
    'LLeida': 'Cataluña',
    'La Rioja': 'La Rioja',
    'Lugo': 'Galicia',
    'Madrid': 'Comunidad de Madrid',
    'Malaga': 'Andalucía',
    'Melilla': 'Melilla',
    'Murcia': 'Región de Murcia',
    'Navarra': 'Navarra',
    'Orense': 'Galicia',
    'Palencia': 'Castilla y León',
    'Las Palmas': 'Canarias',
    'Pontevedra': 'Galicia',
    'Salamanca': 'Castilla y León',
    'Santa Cruz de Tenerife': 'Canarias',
    'Segovia': 'Castilla y León',
    'Sevilla': 'Andalucía',
    'Soria': 'Castilla y León',
    'Tarragona': 'Cataluña',
    'Teruel': 'Aragón',
    'Toledo': 'Castilla-La Mancha',
    'Valencia': 'Comunidad Valenciana',
    'Valladolid': 'Castilla y León',
    'Vizcaya': 'País Vasco',
    'Zamora': 'Castilla y León',
    'Zaragoza': 'Aragón',
    'Ceuta': 'Ceuta'
}

# Normalizar nombres de provincias con errores de codificación
correcciones = {
    'i\x81vila': 'Ávila',
    'i\x81lava': 'Álava',
    'Almeri\xada': 'Almería',
    'Guipiºzcoa': 'Guipúzcoa',
    'La Corui±a': 'La Coruña'
}

df['Provincia'] = df['Provincia'].replace(correcciones)

# Asignar comunidad autónoma en base a la provincia
df['Comunidad'] = df['Provincia'].map(mapa_provincias)


In [ ]:
# Ver valores únicos en la columna 'campanna_anno'
provincias_unicas = df['Provincia'].unique()
provincias_unicas

array(['Segovia', 'Alicante', 'Toledo', 'Leon', 'LLeida', 'Ciudad Real',
       'Ávila', 'Guipúzcoa', 'Vizcaya', 'Orense', 'Castellon',
       'Pontevedra', 'Almería', 'Valladolid', 'Jaen', 'Lugo',
       'Santa Cruz de Tenerife', 'Albacete', 'Huelva', 'Salamanca',
       'Ceuta', 'Baleares', 'Gerona', 'Barcelona', 'Sevilla',
       'Guadalajara', 'Soria', 'Caceres', 'Madrid', 'Cordoba', 'Burgos',
       'Cadiz', 'Navarra', 'Melilla', 'Granada', 'Badajoz', 'Valencia',
       'Tarragona', 'La Coruña', 'Las Palmas', 'Murcia', 'Palencia',
       'La Rioja', 'Teruel', 'Zamora', 'Zaragoza', 'Álava', 'Cuenca',
       'Huesca', 'Malaga'], dtype=object)

In [ ]:
df

,ID,Genero,Casado,Edad,Graduado,Profesion,Experiencia_laboral,Gastoscore,Family_Size,anno_nacimiento,Generacion,Ingresos_anuales_brutos,Provincia,Digital_encuesta,Gasto_medio_mensual_cuota,Abandono,Dias_cliente,Campanna_anno,Comunidad
0,CLI10,F,Si,67,Si,Ingeniero,+20annos,Bajo,3,1957,Generacion X,93442,Segovia,1385,80,1,195,NaN,Castilla y León
1,CLI100,M,Si,64,Si,Medico,+20annos,Medio,2,1960,Generacion X,99774,Alicante,1052,50,1,360,5.0,Comunidad Valenciana
2,CLI1000,M,Si,65,Si,Artista,+20annos,Medio,3,1959,Generacion X,36756,Toledo,1717,30,0,407,3.0,Castilla-La Mancha
3,CLI10000,M,No,33,No,Funcionario,10-20annos,Bajo,4,1991,Millennials,24616,Leon,1606,30,0,544,4.0,Castilla y León
4,CLI10001,M,Si,52,No,Otros,+20annos,Medio,2,1972,Generacion X,29834,LLeida,1346,30,1,170,NaN,Cataluña
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10801,CLI9995,F,Si,76,No,Servicios,+20annos,Alto,2,1948,Generacion X,33389,Toledo,1424,30,1,106,NaN,Castilla-La Mancha
10802,CLI9996,F,No,44,Si,Funcionario,+20annos,Bajo,1,1980,Generacion X,76067,Cordoba,1514,50,0,257,NaN,Andalucía
10803,CLI9997,M,Si,47,Si,Ejecutivo,+20annos,Alto,3,1977,Generacion X,61300,Lugo,1385,80,1,347,5.0,Galicia
10804,CLI9998,M,Si,40,Si,Artista,10-20annos,Alto,3,1984,Millennials,106015,Cuenca,1974,30,0,582,1.0,Castilla-La Mancha


In [ ]:
# Contar valores nulos por columna
valores_nulos = df.isnull().sum()
valores_nulos

ID                              0
Genero                          0
Casado                        190
Edad                            0
Graduado                      103
Profesion                       0
Experiencia_laboral             0
Gastoscore                      0
Family_Size                     0
anno_nacimiento                 0
Generacion                      0
Ingresos_anuales_brutos         0
Provincia                       0
Digital_encuesta                0
Gasto_medio_mensual_cuota       0
Abandono                        0
Dias_cliente                    0
Campanna_anno                5707
Comunidad                       0
dtype: int64

In [ ]:
# Rellenamos los nulos por desconocido
df['Casado'] = df['Casado'].fillna('Desconocido')
df['Graduado'] = df['Graduado'].fillna('Desconocido')

In [ ]:
columnas_numericas = df.select_dtypes(include=['number']).columns
columnas_numericas

Index(['Edad', 'Family_Size', 'anno_nacimiento', 'Ingresos_anuales_brutos',
       'Digital_encuesta', 'Gasto_medio_mensual_cuota', 'Abandono',
       'Dias_cliente', 'Campanna_anno'],
      dtype='object')

In [ ]:
df

,ID,Genero,Casado,Edad,Graduado,Profesion,Experiencia_laboral,Gastoscore,Family_Size,anno_nacimiento,Generacion,Ingresos_anuales_brutos,Provincia,Digital_encuesta,Gasto_medio_mensual_cuota,Abandono,Dias_cliente,Campanna_anno,Comunidad
0,CLI10,F,Si,67,Si,Ingeniero,+20annos,Bajo,3,1957,Generacion X,93442,Segovia,1385,80,1,195,NaN,Castilla y León
1,CLI100,M,Si,64,Si,Medico,+20annos,Medio,2,1960,Generacion X,99774,Alicante,1052,50,1,360,5.0,Comunidad Valenciana
2,CLI1000,M,Si,65,Si,Artista,+20annos,Medio,3,1959,Generacion X,36756,Toledo,1717,30,0,407,3.0,Castilla-La Mancha
3,CLI10000,M,No,33,No,Funcionario,10-20annos,Bajo,4,1991,Millennials,24616,Leon,1606,30,0,544,4.0,Castilla y León
4,CLI10001,M,Si,52,No,Otros,+20annos,Medio,2,1972,Generacion X,29834,LLeida,1346,30,1,170,NaN,Cataluña
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10801,CLI9995,F,Si,76,No,Servicios,+20annos,Alto,2,1948,Generacion X,33389,Toledo,1424,30,1,106,NaN,Castilla-La Mancha
10802,CLI9996,F,No,44,Si,Funcionario,+20annos,Bajo,1,1980,Generacion X,76067,Cordoba,1514,50,0,257,NaN,Andalucía
10803,CLI9997,M,Si,47,Si,Ejecutivo,+20annos,Alto,3,1977,Generacion X,61300,Lugo,1385,80,1,347,5.0,Galicia
10804,CLI9998,M,Si,40,Si,Artista,10-20annos,Alto,3,1984,Millennials,106015,Cuenca,1974,30,0,582,1.0,Castilla-La Mancha


In [ ]:

# Revisión de los tipos de datos únicos por cada columna categórica
for col in df.select_dtypes(include='object').columns:
    tipos = df[col].apply(type).unique()
    print(f"{col}: {tipos}")

ID: [<class 'str'>]
Genero: [<class 'str'>]
Casado: [<class 'str'>]
Graduado: [<class 'str'>]
Profesion: [<class 'str'>]
Experiencia_laboral: [<class 'str'>]
Gastoscore: [<class 'str'>]
Generacion: [<class 'str'>]
Provincia: [<class 'str'>]
Comunidad: [<class 'str'>]


In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Antes de usar LabelEncoder, mira el tipo original:
df['Experiencia_laboral'].value_counts()  # Si son números, NO le hagas label encoding

# Lo mismo para 'Gastoscore'


In [ ]:
# --- Label Encoding para categóricas binarias ---
label_col = ['Genero', 'Casado', 'Graduado', 'Profesion', 'Experiencia_laboral', 'Gastoscore', 'Generacion', 'Abandono', 'Comunidad']
le = LabelEncoder()
for col in label_col:
    df[col] = le.fit_transform(df[col])

In [ ]:
df

,ID,Genero,Casado,Edad,Graduado,Profesion,Experiencia_laboral,Gastoscore,Family_Size,anno_nacimiento,Generacion,Ingresos_anuales_brutos,Provincia,Digital_encuesta,Gasto_medio_mensual_cuota,Abandono,Dias_cliente,Campanna_anno,Comunidad
0,CLI10,0,2,67,2,4,0,1,3,1957,1,93442,Segovia,1385,80,1,195,NaN,3
1,CLI100,1,2,64,2,5,0,2,2,1960,1,99774,Alicante,1052,50,1,360,5.0,7
2,CLI1000,1,2,65,2,0,0,2,3,1959,1,36756,Toledo,1717,30,0,407,3.0,4
3,CLI10000,1,1,33,1,3,1,1,4,1991,3,24616,Leon,1606,30,0,544,4.0,3
4,CLI10001,1,2,52,1,7,0,2,2,1972,1,29834,LLeida,1346,30,1,170,NaN,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10801,CLI9995,0,2,76,1,8,0,0,2,1948,1,33389,Toledo,1424,30,1,106,NaN,4
10802,CLI9996,0,1,44,2,3,0,1,1,1980,1,76067,Cordoba,1514,50,0,257,NaN,0
10803,CLI9997,1,2,47,2,2,0,0,3,1977,1,61300,Lugo,1385,80,1,347,5.0,10
10804,CLI9998,1,2,40,2,0,1,0,3,1984,3,106015,Cuenca,1974,30,0,582,1.0,4


In [ ]:
from sklearn.preprocessing import StandardScaler

# Definir columnas numéricas
columnas_numericas = ['Edad', 'Family_Size', 'anno_nacimiento', 'Ingresos_anuales_brutos',
       'Digital_encuesta', 'Gasto_medio_mensual_cuota', 'Abandono', 'Comunidad',
       'Dias_cliente', 'Campanna_anno', 'Gastoscore', 'Experiencia_laboral', 'Casado', 'Profesion']

# Escalar los datos
scaler = StandardScaler()
df[columnas_numericas] = scaler.fit_transform(df[columnas_numericas])

In [ ]:
df

,ID,Genero,Casado,Edad,Graduado,Profesion,Experiencia_laboral,Gastoscore,Family_Size,anno_nacimiento,Generacion,Ingresos_anuales_brutos,Provincia,Digital_encuesta,Gasto_medio_mensual_cuota,Abandono,Dias_cliente,Campanna_anno,Comunidad
0,CLI10,0,0.833063,1.035176,2,0.268026,-0.775758,-0.151048,0.152186,-1.034957,1,1.052800,Segovia,-0.099417,1.210732,1.534894,-0.692072,NaN,-0.541836
1,CLI100,1,0.833063,0.872409,2,0.623398,-0.775758,1.465015,-0.493738,-0.872187,1,1.231385,Alicante,-0.255354,0.164441,1.534894,0.416520,1.132443,0.319047
2,CLI1000,1,0.833063,0.926665,2,-1.153465,-0.775758,1.465015,0.152186,-0.926444,1,-0.545951,Toledo,0.056052,-0.533086,-0.651511,0.732300,-0.476229,-0.326615
3,CLI10000,1,-1.050611,-0.809517,1,-0.087347,0.470020,-0.151048,0.798110,0.809766,3,-0.888343,Leon,0.004073,-0.533086,-0.651511,1.652767,0.328107,-0.541836
4,CLI10001,1,0.833063,0.221341,1,1.334144,-0.775758,1.465015,-0.493738,-0.221108,1,-0.741176,LLeida,-0.117680,-0.533086,1.534894,-0.860041,NaN,-0.111395
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10801,CLI9995,0,0.833063,1.523477,1,1.689516,-0.775758,-1.767111,-0.493738,-1.523266,1,-0.640912,Toledo,-0.081154,-0.533086,1.534894,-1.290040,NaN,-0.326615
10802,CLI9996,0,-1.050611,-0.212705,2,-0.087347,-0.775758,-0.151048,-1.139662,0.212944,1,0.562762,Cordoba,-0.039009,0.164441,-0.651511,-0.275511,NaN,-1.187499
10803,CLI9997,1,0.833063,-0.049938,2,-0.442719,-0.775758,-1.767111,0.152186,0.050175,1,0.146279,Lugo,-0.099417,1.210732,1.534894,0.329176,1.132443,0.964709
10804,CLI9998,1,0.833063,-0.429728,2,-1.153465,0.470020,-1.767111,0.152186,0.429970,3,1.407404,Cuenca,0.176400,-0.533086,-0.651511,1.908079,-2.084901,-0.326615


In [ ]:
df.to_csv('limpio.csv', index=False)


## 🔍 Interpretación de los Clústeres

> **Nota rápida sobre las medias estandarizadas**  
Los valores que ves no son los reales, sino valores escalados (z-score). Eso significa:
- `0` = promedio general  
- `+` = por encima del promedio  
- `–` = por debajo del promedio  

---

### 🟢 Clúster 0 – Jóvenes digitales con gasto moderado  
📊 **Tamaño:** 3.269 clientes  

- Edad y año de nacimiento ≈ promedio  
- Alta formación: graduado (**1.84**)  
- Experiencia laboral ligeramente baja (**–0.54**)  
- Ingresos y gasto mensual ≈ promedio  
- Uso digital bajo  
- Menor probabilidad de estar casados  
- Comunidad ≈ promedio  

💡 **Interpretación:**  
Clientes jóvenes o de mediana edad, educados, no necesariamente casados, con un perfil económico promedio y uso digital bajo. Representan consumidores estables pero no muy rentables aún.

---

### 🔴 Clúster 1 – Mayores con bajo ingreso y poco digital  
📊 **Tamaño:** 3.975 clientes (el más grande)

- Edad muy baja (**–0.95**) y año de nacimiento muy alto → probablemente personas jóvenes  
- Experiencia laboral más alta de todos (**1.13**)  
- Ingresos bajos (**–0.88**) y gasto mensual también bajo (**–0.52**)  
- Uso digital levemente positivo  
- Más casados de lo esperado (negativo en "Casado" porque codificaste 1=casado, 0=no)  
- Comunidad y generación: valores medios  

💡 **Interpretación:**  
Clientes jóvenes trabajadores, quizás recién independizados, con bajo poder adquisitivo. Usan un poco más los canales digitales. Son muchos, pero tienen bajo potencial económico actualmente.

---

### 🔷 Clúster 2 – Ricos gastadores y digitales  
📊 **Tamaño:** 1.106 clientes (el más pequeño)

- Ingresos altos (**0.80**) y gasto mensual muy alto (**1.00**)  
- Edad alta (**1.86**) y año de nacimiento bajo (**–1.86**) → adultos mayores  
- Baja experiencia laboral (**–0.77**)  
- Comunidad: baja  
- Digitalidad: baja, lo cual sorprende  
- Mayor formación  

💡 **Interpretación:**  
Clientes mayores con alto poder adquisitivo que gastan mucho. Aunque son pocos, tienen gran valor económico. Podrían ser retirados con buenos ahorros o rentistas. No muy digitales, podrían requerir atención más tradicional.

---

### 🟡 Clúster 3 – Adultos medios, equilibrados y con poder  
📊 **Tamaño:** 2.456 clientes

- Edad y año de nacimiento ligeramente por encima del promedio  
- Ingresos altos (**0.80**) y gasto alto (**0.56**)  
- Experiencia laboral baja (**–0.76**)  
- Comunidad: ligeramente negativa  
- Formación y digitalización en niveles intermedios  

💡 **Interpretación:**  
Clientes adultos, con buen nivel de ingresos y gasto. Son equilibrados, no tan mayores como el Clúster 2, ni tan jóvenes como el 1. Buen target para fidelización digital o promociones segmentadas.


## 🎯 Recomendaciones Estratégicas por Clúster

---

### 🟢 Clúster 0 – Jóvenes digitales con gasto moderado

🔧 **Estrategias sugeridas:**

- **Educación financiera y cross-selling:** Ofrecer productos financieros escalables o formativos que los acompañen en su crecimiento económico (cuentas ahorro, microinversiones, planes a futuro).
- **Incentivar el uso digital:** Promociones o beneficios por uso de app/web, ya que su nivel de digitalización es bajo.
- **Campañas basadas en estilo de vida:** Comunicación fresca, moderna y no tradicional, dirigida por redes sociales o influencers.
- **Fomentar la fidelización temprana:** Están en un punto donde aún no son muy rentables, pero pueden crecer con la marca si se cultiva su lealtad.

---

### 🔴 Clúster 1 – Jóvenes trabajadores con bajo ingreso y poco digital

🔧 **Estrategias sugeridas:**

- **Ofertas básicas y económicas:** Productos con bajo coste o sin comisiones, cuentas sin mantenimiento, promociones de captación.
- **Educación digital:** Formación sobre el uso de apps, atención automatizada o contenidos de ayuda para navegar productos digitales.
- **Segmentación local:** Aprovechar la alta cantidad para hacer campañas regionales de impacto alto a bajo coste.
- **Propuestas laborales o de networking:** Vinculación a programas para jóvenes o empleo que refuercen la experiencia de marca.

---

### 🔷 Clúster 2 – Ricos, mayores y poco digitales

🔧 **Estrategias sugeridas:**

- **Atención personalizada:** Servicios premium, gestores personales o canales no digitales (teléfono, presencial).
- **Productos de inversión y seguros:** Enfocar en productos patrimoniales o planes de retiro, ya que tienen alto gasto e ingresos.
- **Comunicación clara y directa:** Evitar tecnicismos o canales digitales forzados.
- **Upselling con asesoría humana:** Promover productos más rentables mediante acompañamiento cercano.

---

### 🟡 Clúster 3 – Adultos medios, equilibrados y con poder

🔧 **Estrategias sugeridas:**

- **Campañas de fidelización:** Programas de puntos, recompensas por antigüedad, experiencias personalizadas.
- **Ofertas digitales bien dirigidas:** Son usuarios intermedios, por lo que se puede potenciar aún más la digitalización con UX optimizada.
- **Segmentación por estilo de vida:** Adaptar los mensajes a perfiles adultos con responsabilidades (hogar, hijos, viajes, inversiones).
- **Productos combinados:** Bundles que combinen servicios digitales, seguros o beneficios familiares.

---

## 📌 Recomendaciones Generales

- **Personalización basada en cluster:** Automatizar campañas y journeys por segmentos, integrando modelos de predicción de abandono o valor.
- **Monitoreo continuo:** Repetir el análisis regularmente para detectar cambios en comportamiento o necesidades.
- **Enriquecimiento de datos:** Incorporar variables externas (geográficas, digitales, consumo en ecommerce) para mejorar segmentación.
- **Formación al equipo comercial:** Capacitar en cómo abordar a cada perfil, especialmente en canales físicos o mixtos.

